In [1]:
import numpy as np
import pandas as pd

This will be the notebook in which my initial analysis for the amazon data set takes place. I will use the smaller sample dataset I created, and go through the whole analysis with this dataset to iron out my methodology. 

In [2]:
sample_df = pd.read_csv(r'rough_data\amazon_dataset\amzn_sample.csv')

In [3]:
sample_df

,item,rating
0,0060721545,5.0
1,0060721545,2.0
2,0060721545,2.0
3,0060721545,1.0
4,0060721545,3.0
...,...,...
92404,B00J5364J4,4.0
92405,B00J5364J4,5.0
92406,B00J5364J4,1.0
92407,B00J5364J4,5.0


There are 92,409 reviews in my dataset

In [55]:
sample_df['rating'].value_counts()

5.0    54959
4.0    17684
3.0     8042
1.0     6873
2.0     4851
Name: rating, dtype: int64

In [56]:
sample_df['rating'].value_counts().sum()

92409

The above two calculations ensure that all ratings are integer values between 1 and 5

In [4]:
sample_df['item'].value_counts()

043935806X    4683
B006Z48TZS    3680
B002AQHLEU    3128
B008LFTCAK    2884
B008K6G8CK    2375
              ... 
B0074PFIKQ      50
B001F7E9VI      50
B000RL0TE4      50
B005G80JB6      50
B008AP09XA      50
Name: item, Length: 500, dtype: int64

There are indeed 500 unique items in my dataset (previously known, since this was determined from my construction of the sample data).

In [62]:
# Create the structure of the final dataframe
grouped_df = pd.DataFrame(sample_df['item'].unique(), columns=['item'])
# Create a series for number of reviews in each rating category
for star in range(1,6):
    grouped_df = pd.merge(grouped_df, sample_df[sample_df['rating'] == star].groupby('item').count(), how='left', left_on='item', right_index=True)
    grouped_df = grouped_df.rename(columns={'rating': f'{star}_star_ratings'})

# Form a total number of ratings column and turn raw ratings counts into distributions.
grouped_df['total_ratings'] = (grouped_df['1_star_ratings'] + grouped_df['2_star_ratings'] 
                            + grouped_df['3_star_ratings'] + grouped_df['4_star_ratings'] 
                            + grouped_df['5_star_ratings'])


grouped_df

,item,1_star_ratings,2_star_ratings,3_star_ratings,4_star_ratings,5_star_ratings,total_ratings
0,0060721545,24.0,8.0,7.0,15,17,71.0
1,0060959258,1.0,6.0,14.0,33,53,107.0
2,0061834785,2.0,8.0,25.0,35,38,108.0
3,006195828X,1.0,3.0,8.0,9,32,53.0
4,0062069403,13.0,23.0,16.0,5,17,74.0
...,...,...,...,...,...,...,...
495,B00HVHX11K,7.0,6.0,7.0,15,33,68.0
496,B00IBR189Q,8.0,4.0,1.0,11,48,72.0
497,B00IE4EIW0,7.0,6.0,6.0,31,90,140.0
498,B00ILI95HW,3.0,7.0,13.0,34,99,156.0


### Checks to make sure all the data came through correctly

In [83]:
#Test that all the values came through and match original table.
print('1-star test:', sample_df['rating'].value_counts()[1.0] == grouped_df['1_star_ratings'].sum())
print('2-star test:', sample_df['rating'].value_counts()[2.0] == grouped_df['2_star_ratings'].sum())
print('3-star test:', sample_df['rating'].value_counts()[3.0] == grouped_df['3_star_ratings'].sum())
print('4-star test:', sample_df['rating'].value_counts()[4.0] == grouped_df['4_star_ratings'].sum())
print('5-star test:', sample_df['rating'].value_counts()[5.0] == grouped_df['5_star_ratings'].sum())
print('total test:', sample_df['rating'].value_counts().sum() == grouped_df['total_ratings'].sum())




1-star test: True
2-star test: True
3-star test: True
4-star test: True
5-star test: True
total test: False


## The data did not come through the grouping correctly. I need to figure out what happened.
This is a sneaky error, it didn't break my program or show any hints of being present until I did a check to make sure none of the data was lost while grouping. If I didn't do the check, I would not have caught this until much later in the analysis.

In [78]:
#Proof no null values are present. In original data.
np.any(sample_df.isnull(), axis=0)

item      False
rating    False
dtype: bool

In [82]:
#Test for null values in grouped data.
grouped_df[np.any(grouped_df.isnull(),axis=1)]

,item,1_star_ratings,2_star_ratings,3_star_ratings,4_star_ratings,5_star_ratings,total_ratings
5,0062104187,1.0,NaN,2.0,11,87,NaN
7,0143118765,NaN,2.0,8.0,18,40,NaN
9,0262541157,NaN,NaN,2.0,14,50,NaN
17,0345517830,NaN,2.0,6.0,24,38,NaN
19,0373835639,NaN,4.0,5.0,14,84,NaN
...,...,...,...,...,...,...,...
480,B00ED2TGZ6,NaN,3.0,7.0,10,44,NaN
488,B00GD3CHMM,NaN,1.0,4.0,17,37,NaN
490,B00GUNAJ66,NaN,NaN,1.0,7,72,NaN
493,B00HQ4W1QE,NaN,1.0,2.0,5,47,NaN


My theory is that, if there are no ratings in a category for an item, the field is filled in with NaN rather than 0. I will inspect this for a few of the rows identified to have NaN values.



In [105]:
t_item = grouped_df.iloc[5,0]
print('Test for item', t_item)
print('1-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 1]['item'].count())
print('2-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 2]['item'].count())
print('3-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 3]['item'].count())
print('4-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 4]['item'].count())
print('5-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 5]['item'].count())
print('total reviews:', sample_df[sample_df['item'] == t_item]['item'].count())

Test for item 0062104187
1-star reviews: 1
2-star reviews: 0
3-star reviews: 2
4-star reviews: 11
5-star reviews: 87
total reviews: 101


<ipython-input-105-e2aeecc040e8>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('1-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 1]['item'].count())
<ipython-input-105-e2aeecc040e8>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('2-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 2]['item'].count())
<ipython-input-105-e2aeecc040e8>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('3-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 3]['item'].count())
<ipython-input-105-e2aeecc040e8>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('4-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 4]['item'].count())
<ipython-input-105-e2aeecc040e8>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [106]:
t_item = grouped_df.iloc[490,0]
print('Test for item', t_item)
print('1-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 1]['item'].count())
print('2-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 2]['item'].count())
print('3-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 3]['item'].count())
print('4-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 4]['item'].count())
print('5-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 5]['item'].count())
print('total reviews:', sample_df[sample_df['item'] == t_item]['item'].count())

Test for item B00GUNAJ66
1-star reviews: 0
2-star reviews: 0
3-star reviews: 1
4-star reviews: 7
5-star reviews: 72
total reviews: 80


<ipython-input-106-43621ddf1d88>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('1-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 1]['item'].count())
<ipython-input-106-43621ddf1d88>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('2-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 2]['item'].count())
<ipython-input-106-43621ddf1d88>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('3-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 3]['item'].count())
<ipython-input-106-43621ddf1d88>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('4-star reviews:', sample_df[sample_df['item'] == t_item][ sample_df['rating'] == 4]['item'].count())
<ipython-input-106-43621ddf1d88>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


This is indeed what is happening. There is an easy fix for this, given where and why the NaN is arising. I can fill the NaN's with 0 during the generation of the star columns, and this hould take care of all my issues. Zero is the appropriate number because the NaN arises when there is nothing to count across. This means there were zero reviews for that specific item/rating-value combination. 

I will redo the calculations from above relating to data generation to prove the problem is fixed and to preserve the record of this problem having arisen.

In [6]:
# FIXING MY DATAFRAME GENERATION TO ACCOUNT FOR NaN VALUES

# Create the structure of the final dataframe
grouped_df = pd.DataFrame(sample_df['item'].unique(), columns=['item'])
# Create a series for number of reviews in each rating category
for star in range(1,6):
    grouped_df = pd.merge(grouped_df, sample_df[sample_df['rating'] == star].groupby('item').count(), how='left', left_on='item', right_index=True)
    grouped_df = grouped_df.fillna(0)
    grouped_df = grouped_df.rename(columns={'rating': f'{star}_star_ratings'})

# Form a total number of ratings column and turn raw ratings counts into distributions.
grouped_df['total_ratings'] = (grouped_df['1_star_ratings'] + grouped_df['2_star_ratings'] 
                            + grouped_df['3_star_ratings'] + grouped_df['4_star_ratings'] 
                            + grouped_df['5_star_ratings'])

# Also decided I want an average rating column.
grouped_df['average_rating'] = (1*grouped_df['1_star_ratings'] + 2*grouped_df['2_star_ratings'] 
                            + 3*grouped_df['3_star_ratings'] + 4*grouped_df['4_star_ratings'] 
                            + 5*grouped_df['5_star_ratings']) / grouped_df['total_ratings']

grouped_df

,item,1_star_ratings,2_star_ratings,3_star_ratings,4_star_ratings,5_star_ratings,total_ratings,average_rating
0,0060721545,24.0,8.0,7.0,15,17,71.0,2.901408
1,0060959258,1.0,6.0,14.0,33,53,107.0,4.224299
2,0061834785,2.0,8.0,25.0,35,38,108.0,3.916667
3,006195828X,1.0,3.0,8.0,9,32,53.0,4.283019
4,0062069403,13.0,23.0,16.0,5,17,74.0,2.864865
...,...,...,...,...,...,...,...,...
495,B00HVHX11K,7.0,6.0,7.0,15,33,68.0,3.897059
496,B00IBR189Q,8.0,4.0,1.0,11,48,72.0,4.208333
497,B00IE4EIW0,7.0,6.0,6.0,31,90,140.0,4.364286
498,B00ILI95HW,3.0,7.0,13.0,34,99,156.0,4.403846


## Now, Testing the data I believe was corrected.

In [7]:
#Test that all the values came through and match original table.
print('1-star test:', sample_df['rating'].value_counts()[1.0] == grouped_df['1_star_ratings'].sum())
print('2-star test:', sample_df['rating'].value_counts()[2.0] == grouped_df['2_star_ratings'].sum())
print('3-star test:', sample_df['rating'].value_counts()[3.0] == grouped_df['3_star_ratings'].sum())
print('4-star test:', sample_df['rating'].value_counts()[4.0] == grouped_df['4_star_ratings'].sum())
print('5-star test:', sample_df['rating'].value_counts()[5.0] == grouped_df['5_star_ratings'].sum())
print('total test:', sample_df['rating'].value_counts().sum() == grouped_df['total_ratings'].sum())



1-star test: True
2-star test: True
3-star test: True
4-star test: True
5-star test: True
total test: True


It worked! Moving on to the next part of teh analysis (after formatting the table I have).

Getting the data in the final format that I want it.

In [8]:
# Format the data how I want it.
grouped_df['1_star_ratings'] = grouped_df['1_star_ratings'] / grouped_df['total_ratings']
grouped_df['2_star_ratings'] = grouped_df['2_star_ratings'] / grouped_df['total_ratings']
grouped_df['3_star_ratings'] = grouped_df['3_star_ratings'] / grouped_df['total_ratings']
grouped_df['4_star_ratings'] = grouped_df['4_star_ratings'] / grouped_df['total_ratings']
grouped_df['5_star_ratings'] = grouped_df['5_star_ratings'] / grouped_df['total_ratings']

#Reorder columns for maximum usefulness
grouped_df = grouped_df[['item', 'average_rating', 'total_ratings', '5_star_ratings', '4_star_ratings', \
                       '3_star_ratings', '2_star_ratings', '1_star_ratings']]

#This is my complete data set.
grouped_df

,item,average_rating,total_ratings,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings
0,0060721545,2.901408,71.0,0.239437,0.211268,0.098592,0.112676,0.338028
1,0060959258,4.224299,107.0,0.495327,0.308411,0.130841,0.056075,0.009346
2,0061834785,3.916667,108.0,0.351852,0.324074,0.231481,0.074074,0.018519
3,006195828X,4.283019,53.0,0.603774,0.169811,0.150943,0.056604,0.018868
4,0062069403,2.864865,74.0,0.229730,0.067568,0.216216,0.310811,0.175676
...,...,...,...,...,...,...,...,...
495,B00HVHX11K,3.897059,68.0,0.485294,0.220588,0.102941,0.088235,0.102941
496,B00IBR189Q,4.208333,72.0,0.666667,0.152778,0.013889,0.055556,0.111111
497,B00IE4EIW0,4.364286,140.0,0.642857,0.221429,0.042857,0.042857,0.050000
498,B00ILI95HW,4.403846,156.0,0.634615,0.217949,0.083333,0.044872,0.019231


I want to save this dataset as the fully-transformed data that I will use in my analysis and modeling.

In [9]:
# Commented out this line to avoid accidental overwriting of the file
# grouped_df.to_csv(r'rough_data\amazon_dataset\amzn_sample_final_format.csv', index=False)

This went so well that I think I will transform the rest of my full dataset before moving on to the next step of my analysis.